In [143]:
from scipy.sparse import csr_matrix
import numpy as np
raw_file = open("ml-1m.tsv", "r")
lines = raw_file.readlines()

In [144]:
edge = [[], []]
label = []
for line in lines:
    line = line.strip("/n")
    line = line.split("\t")
    fr, to, sign = line
    edge[0].append(int(fr))
    edge[1].append(int(to))
    label.append(sign)


In [145]:
num_user = max(edge[0])+1
num_user

6040

In [146]:
num_item = max(edge[1])+1
num_item

3952

In [147]:
num_nodes = num_item+num_user

In [148]:
import torch
A = torch.sparse_coo_tensor(torch.tensor(edge), torch.tensor([1]*len(edge[0])), (num_nodes, num_nodes))
A =  torch.eye(num_nodes) + A 
A.shape

torch.Size([9992, 9992])

In [149]:
row_sum = torch.sum(A, dim=1)

In [150]:
d_inv_row = 1.0 / row_sum.to_dense()
d_inv_row[torch.isinf(d_inv_row)] = 0
d_inv_matrix = torch.diag(d_inv_row.to_dense())


In [151]:
d_inv_matrix

tensor([[0.0185, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0077, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0192,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 1.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0000]])

In [152]:
A = A.to(torch.float32)

In [161]:
nA = torch.sparse.mm(d_inv_matrix,A)

In [162]:
nAT = nA.T.to("cuda:0")

In [163]:
torch.sum(nAT, dim=0)

tensor([1., 1., 1.,  ..., 1., 1., 1.], device='cuda:0')

In [169]:
alpha = 0.8
x0 = torch.randn(num_nodes, num_nodes).abs().to("cuda:0")
I = torch.eye(num_nodes).to("cuda:0")
x = x0.to("cuda:0")
for i in range(100):
    x = (1-alpha) * nAT.matmul(x) + alpha * I

In [170]:
x

tensor([[2.0790e-01, 1.7321e-03, 1.8093e-03,  ..., 2.8121e-09, 2.8209e-09,
         2.7936e-09],
        [2.7806e-04, 2.0151e-01, 3.7871e-04,  ..., 4.6816e-10, 4.6962e-10,
         4.6507e-10],
        [3.2237e-04, 3.4595e-04, 2.0350e-01,  ..., 5.5000e-10, 5.5172e-10,
         5.4637e-10],
        ...,
        [1.3548e-10, 3.1371e-10, 9.6911e-11,  ..., 1.0000e+00, 3.2088e-10,
         8.1092e-11],
        [2.5457e-10, 1.9706e-11, 1.4497e-10,  ..., 8.9851e-11, 1.0000e+00,
         2.9434e-11],
        [1.6471e-10, 1.6224e-10, 2.7731e-10,  ..., 8.5831e-11, 3.6266e-10,
         1.0000e+00]], device='cuda:0')

In [171]:
torch.sum(x, dim=0)

tensor([1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000], device='cuda:0')